# Handling multi-dimensional arrays with xarray

## Authors & Contributors

### Authors

- Pier Lorenzo Marasco, Ispra (Italy), [@pl-marasco](https://github.com/pl-marasco)

### Contributors

- Alejandro Coca-Castro, The Alan Turing Institute (United Kingdom), [@acocac](https://github.com/acocac)
- Anne Fouilloux, University of Oslo (Norway), [@annefou](https://github.com/annefou)
- Guillaume Eynard-Bontemps, CNES (France), [@guillaumeeb](https://github.com/guillaumeeb)

<div class="alert alert-info">
    <i class="fa-question-circle fa" style="font-size: 22px;color:#666;"></i> <b>Overview</b>
    <br>
    <br>
    <b>Questions</b>
    <ul>
        <li>What is Xarray?</li>
        <li>How to open a local file?</li>
        <li>How to print metadata information?</li>
        <li>How to make a selection?</li>
        <li>How to visualize with matplotlib?</li>
        <li>How to perform basic computations, statistics and aggregations?</li>
        <li>How to mask data?</li>
    </ul>
    <b>Objectives</b>
    <ul>
        <li>Learn about Xarray Python ecosystem</li>
        <li>Learn file handling with xarray</li>
        <li>Learn to get metadata information</li>
        <li>Learn to select and mask data</li>
        <li>Learn to make basic computations, aggregations and statistics</li>
        <li>Learnn to visualize data</li>
    </ul>
</div>

## Context

We will be using [Pangeo](https://pangeo.io/) open-source software stack for computing and visualizing the Vegetation Condition Index (VCI) {cite:ps}`kogan1995-VCI`, a well-established indicator to estimate droughts from remote sensing data.

VCI compares the current normalized difference vegetation index (NDVI) {cite:ps}`wiki2022-ndvi` to the range of values observed in previous years.

### Data

In this episode, we will use Sentinel-3 NDVI Analysis Ready Data (ARD) provided by the [Copernicus Global Land Service](https://land.copernicus.eu/global/index.html).

This dataset can be discovered through the [OpenEO API](https://openeo.org/) from the CGLS distributor, [VITO](https://vito.be/en/). Access is free of charge but an [EGI registration](https://aai.egi.eu/) is needed.

The same dataset can also be downloaded from [Zenodo](https://zenodo.org/): [FOSS4G Training Datasets: NDVI](https://zenodo.org/record/6969999/files/C_GLS_NDVI_20220101_20220701_Lombardia_S3_2.nc)

Further info about drought indexes can be found in the Integrated Drought Management Programme (see [here](https://www.droughtmanagement.info/indices/)).

## Setup

This episode uses the following Python packages:

- xarray
- netcdf4
- h5netcdf
- wget
- numpy

Please install these packages if not already available in your Python environment (see [Setup page](https://pangeo-data.github.io/foss4g-2022/before/setup.html)). Below, we only install packages that are not available in the EGI-ACE EOSC deployment of Pangeo for the FOSS4G course.

### Packages

In this episode, Python packages are imported when we start to use them. However, for best software practices, we recommend you to install and import all the necessary libraries at the top of your Jupyter notebook.

In [ ]:
import xarray as xr

### Fetch Data

- For now we will fetch a netCDF file containing Sentinel-3 NDVI Analysis Ready Data (ARD).
- The file is available in a Zenodo repository. We will download it using using `pooch`, a very handy Python-based library to download and cache your data files locally (see further info [here](https://www.fatiando.org/pooch/latest/index.html))
- Later in this episode, we will learn about different ways to access data, including access to remote data.

In [ ]:
import pooch

In [ ]:
cgls_file = pooch.retrieve(
    url="https://zenodo.org/record/6969999/files/C_GLS_NDVI_20220101_20220701_Lombardia_S3_2.nc",
    known_hash="md5:bbb25f1865056c886c6f9b37147d8f2f",
    path=f".",
)

## Open and read metadata through Xarray

In [ ]:
cgls_ds = xr.open_dataset(cgls_file)

As the dataset is in NetCDF format Xarray automatically selects the correct engine (this underneath has been
conducted through the automatic specification of engine='netcdf'). If the dataset is in geoTiff format the engine would be 'rasterio'.
Supposing that you have a dataset in an unrecognised format you can always create your own engine and pass it through the engine parameter.

:::{tip}
If you get an error with the previous command, first check the location of the input file some_hash-C_GLS_NDVI_20220101_20220701_Lombardia_S3_2.nc: it should have been downloaded in the same directory as your Jupyter Notebook.
:::

In [ ]:
cgls_ds

## What is xarray?

Xarray introduces labels in the form of dimensions, coordinates and attributes on top of raw NumPy-like multidimensional arrays, which allows for a more intuitive, more concise, and less error-prone developer experience.

### How is xarray structured?

Xarray has two core data structures, which build upon and extend the core strengths of NumPy and Pandas libraries. Both data structures are fundamentally N-dimensional:

- [DataArray](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.html#xarray.DataArray) is the implementation of a labeled, N-dimensional array. It is an N-D generalization of a pandas.Series. The name DataArray itself is borrowed from Fernando Perez’s datarray project, which prototyped a similar data structure.

- [Dataset](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html#xarray.Dataset) is a multi-dimensional, in-memory array database. It is a dict-like container of DataArray objects aligned along any number of shared dimensions, and serves a similar purpose in xarray to the pandas.DataFrame.


## Accessing Coordinates and Data Variables 

DataArray, within Datasets, can be accessed through:
- the dot notation like Dataset.NameofVariable  
- or using square brackets like Dataset['NameofVariable'] (NameofVariable needs to be a string so use quotes or double quotes)

In [ ]:
cgls_ds.t

`cgls_ds.t` is a `xarray.DataArray` with dates of type `datetime64[ns]`

In [ ]:
cgls_ds.NDVI

In [ ]:
cgls_ds['NDVI']

Same can be achieved for attributes and a DataArray.attrs will return a dictionary.

In [ ]:
cgls_ds['NDVI'].attrs

### Xarray and Memory usage
Memory must be always taken into account. Each DataArray has an impact on it and to avoid issues like killing a notebook Kernel or Python process crash, users must avoid to open data bigger than memory.
If the dataset isn't fitting in the available memory then the best option would be to load it through the help of Dask in a lazy approach. Later on in the tutorial we will introduce this concept.

As the size of the data isn't too big here, we can continue without any problem.

In [ ]:
import numpy as np

In [ ]:
print(f'{np.round(cgls_ds.NDVI.nbytes / 1024**2, 2)} MB') # all the data are automatically loaded into memory when calling xr.open_dataset

## Renaming Coordinates and Data Variables 

As later on other datasets have dimensions named according to the more common triad lat,lon,time a renomination is needed.

In [ ]:
cgls_ds.rename(x='lon', y='lat', t='time')

In [ ]:
cgls_ds = cgls_ds.rename(x='lon', y='lat', t='time')

In [ ]:
cgls_ds

## Selection methods

As underneath DataArrays are Numpy Array objects (that implements the standard Python x[obj] (x: array, obj: int,slice) syntax) data can be accessed through the same approach of numpy indexing.

In [ ]:
cgls_ds.NDVI[0,100,100]

or slicing

In [ ]:
cgls_ds.NDVI[0:5,100:110,100:110]

As it isn't easy to remember the order of dimensions, Xarray really helps by making possible to select the position using names:

- `.isel` -> selection based on positional index
- `.sel`  -> selection based on coordinate values

We first check the number of elements in each coordinate of the NDVI Data Variable using the built-in method sizes. Same result can be achieved querying each coordinate using the Python built-in function `len`.

In [ ]:
cgls_ds.sizes

In [ ]:
cgls_ds.NDVI.isel(time=0, lat=100, lon=100) # same as cgls_ds.NDVI[0,100,100]

The more common way to select a point is through the labeled coordinate using the `.sel` method.

In [ ]:
cgls_ds.NDVI.sel(time='2022-01-01')

Time is easy to be used as there is a 1 to 1 correspondence with values in the index, float values are not that easy to be use and a small difference makes a big difference in results.


Coordinates are always affected by precision issue so the best option would be to use the sampling method that will search for the closest point according to the method requested.

Option for the method are:
- pad / ffill: propagate last valid index value forward
- backfill / bfill: propagate next valid index value backward
- nearest: use nearest valid index value

Another important parameter that can be set is the tolerance that specify the distance between original and the target. (abs(index\[indexer] - target) <= tolerance) from [documentation](https://xarray.pydata.org/en/v0.17.0/generated/xarray.DataArray.sel.html#:~:text=xarray.DataArray.sel%20%C2%B6%20DataArray.sel%28indexers%3DNone%2C%20method%3DNone%2C%20tolerance%3DNone%2C%20drop%3DFalse%2C%20%2A%2Aindexers_kwargs%29%20%C2%B6,this%20method%20should%20use%20labels%20instead%20of%20integers.).

In [ ]:
cgls_ds.sel(lat=46.3, lon=8.8, method='nearest')

:::{warning}
To select a single real value without specifying a method, you would need to specify the exact encoded value; not the one you see when printed.
:::

In [ ]:
cgls_ds.isel(lon=100).lon.values.item()

In [ ]:
cgls_ds.isel(lat=100).lat.values.item()

In [ ]:
cgls_ds.sel(lat=46.336112857142965, lon=8.799356142858498)

That is why we use a `method`! It makes your life easier to deal with inexact matches.

As the exercise is focused on an Area Of Interest, this can be obtained through a bounding box defined with slices.

In [ ]:
NDVI_AOI = cgls_ds.NDVI.sel(lat=slice(46.5,44.5), lon=slice(8.5,11.5))
NDVI_AOI

:::{tip}
Have you noticed that latitudes are selected from the largest to the smallest values e.g. `46.5`, `44.5` while longitudes are selected from the smallest to the largest value e.g. `8.5`,`11.5`.
**The reason is that you need to use the same order than the corresponding DataArray**.
:::

## Plotting
Plotting data can be easily obtained through matplotlib.pyplot back-end [matplotlib documentation](https://matplotlib.org/stable/index.html)

In [ ]:
NDVI_AOI.isel(time=0).plot(cmap="RdYlGn")

In the next episode we will learn more about visualization and how to make interactive plots using [holoviews](https://holoviews.org/).

NDVI values are a little odd in comparison with standard NDVI range values \[-1, +1]. This finds confirmation on the max values reported in the [Product User Manual](https://land.copernicus.eu/global/sites/cgls.vito.be/files/products/CGLOPS1_PUM_NDVI300m-V2_I1.20.pdf) (PUM).

## NDVI characteristics from the Product User Manual (PUM)

| layer name  | description                             | physical min | physical max | digital max | scaling | offset | No Data  |
|-------------|-----------------------------------------|--------------|--------------|-------------|---------|--------|----------|
| ndvi        | normalized difference vegetation index  | -0.08        | 0.92         | 250         | 1/250   | -0.08  | 254, 255 |
| ndvi_unc    | uncertainty on ndvi                     | 0            | 1            | 1000        | 1/1000  | 0      | 65535    |
| nobs        | number of observations                  | 0	           | 32           | 32          | 1       | 0      | 255      |
| qflag       | bitwise quality flag                    | -            | -            | 254         | 1       | 0      | 255      |


from : [Copernicus Global Land Service NDVI 300 V2.0.1](https://land.copernicus.eu/global/sites/cgls.vito.be/files/products/CGLOPS1_PUM_NDVI300m-V2_I1.20.pdf)

## Basic Computations

- Arithmetics
- Reductions

Simple arithmetic can be obtained without worrying about dimensions and coordinates, using the same notation we use with `numpy`.

In [ ]:
NDVI_AOI = NDVI_AOI* (1/250) - 0.08

In [ ]:
NDVI_AOI

## Statistics

All the standard statistic operations can be used such as `min`, `max`, `mean`. When no argument is passed to the function, the operation is done over all the dimension of the variable (same as with `numpy`).

In [ ]:
NDVI_AOI.min()

You can make a statistical operation over a dimension, for instance here to get the maximum value for each available time.

In [ ]:
NDVI_AOI.max(dim='time')

## Aggregation

We have data every 10 days. To obtain monthly values, we can group values per month and take the mean.

In [ ]:
NDVI_monthly = NDVI_AOI.groupby(NDVI_AOI.time.dt.month).mean()

In [ ]:
NDVI_monthly

As we have data from January to July, the time dimension is now `month` and takes values from `1` to `7`.

In [ ]:
NDVI_monthly.month

## Mask

Not all values are valid and masking all the ones not in the valid range \[-0.08, 0.92] is necessary. Masking can be achieved through the method `DataSet|Array.where(cond, other)` or `xr.where(cond, x, y)`.

The difference consists in the possibility to specify the value in case the condition is positive or not; `.where` doesn't give this possibility.

In [ ]:
NDVI_masked = NDVI_AOI.where((NDVI_AOI >= -0.08) & (NDVI_AOI <= 0.92)) # by default where condition is false values are set to NA

In [ ]:
NDVI_masked.isel(time=0).plot()

To better visualize the mask with the help of `xr.where` a specific variable can be constructed. We will only have two values 0 or 1.

In [ ]:
mask = xr.where((NDVI_AOI <= -0.08) | (NDVI_AOI >= 0.92), 1, 0)

In [ ]:
mask.isel(time=0).plot()

Plot a single point over the time dimension

In [ ]:
NDVI_masked.sel(lat=45.88, lon=8.63, method='nearest').plot()

## Save xarray Dataset

It is very often convenient to save intermediate or final results into a local file. We will learn more about the different file formats Xarray can handle but for now let's save it as a netCDF file. Check the file size after saving the result into netCDF.

In [ ]:
NDVI_masked.to_netcdf('C_GLS_NDVI_20220101_20220701_Lombardia_S3_2_masked.nc')

<div class="alert alert-success">
    <i class="fa-check-circle fa" style="font-size: 22px;color:#666;"></i> <b>Key Points</b>
    <br>
    <ul>
        <li>Xarray Dataset and DataArray</li>
        <li>Read and get metadata from local raster file</li>
        <li>Dataset and DataArray selection</li>
        <li>Aggregation and statistics</li>
        <li>Masking values</li>
    </ul>
</div>

## References

```{bibliography}
:style: alpha
:filter: topic % "xarray"
```